### ======================================
<h3 style="color:pink">Import neccesory libraries</h3>

In [ ]:
import pandas as pd
import numpy  as np 
import random
import talib
from lightweight_charts import JupyterChart


### ======================================
<h3 style="color:yellow">Globals</h3>

In [ ]:
backtesterPositions  = []

backtesterType       = 'TwoWay'           # 'OneWay'

backtestercommision  = 0.01               # Half of this commision will apply when opening, and the other half will apply when closing the trade.

backtesterModet      = 'CandleStickBase'  #'TickBase'    #'CandleTickBase'

### ======================================
<h3 style="color:green">Candlestick and Tick and CandleTick Class</h3>

Three classes for BackTest modes.

1) CandleStick class for CandleStick based BackTesting: Fast, but very in complicated situations can become inaccurate. use when very accurate spreads calculations doesn't matter too much.
3) Tick class for Tick based BackTesting: Very slow and has extremely heavy calculation, but the most accurate.
2) CandleTick class for CandleTick based BackTesting: Slow and has heavy calculation, but accurate. use when accurate spreads calculations matters.

Tick data contains detailed information about every individual trade that occurs

In [ ]:
class CandleStick:

    def __init__(self, candleTimestampOpened, candleTimestampClosed, candleIndex, candleOpen, candleHigh, candleLow, candleClose, candleVolume, candleSpread, marketPoints):

        self.index = candleIndex


        self.timestampOpened = candleTimestampOpened
        self.timestampClosed = candleTimestampClosed


        self.open   = candleOpen
        self.high   = candleHigh
        self.low    = candleLow
        self.close  = candleClose
        self.volume = candleVolume


        self.spread   = (candleSpread)
        self.slippage = (self.spread*marketPoints)
    #
#


class Tick:

    def __init__(self, tickTimestamp, tickIndex, tickPrice, tickVolume, bidPrice, askPrice, bidVolume, askVolume, marketPoints):

        self.index = tickIndex


        self.timestamp = tickTimestamp


        self.tradedprice = tickPrice
        self.volume      = tickVolume

        self.askPrice  = askPrice
        self.bidPrice  = bidPrice
        self.askVolume = askVolume
        self.bidVolume = bidVolume
    #
#


class CandleTick:

    def __init__(self, candleTimestampOpened, candleTimestampClosed, candleIndex, candleOpen, candleHigh, candleLow, candleClose, candleTicks: np.array, candleSpread, marketPoints):

        self.candleIndex = candleIndex


        self.timestampOpened = candleTimestampOpened
        self.timestampClosed = candleTimestampClosed


        self.open  = candleOpen
        self.high  = candleHigh
        self.low   = candleLow
        self.close = candleClose


        self.ticks = candleTicks


        self.spread   = (candleSpread)
        self.slippage = (self.spread*marketPoints)
    #
#

### ======================================
<h3 style="color:green">Position Class</h3>

In two-way accounts, every trade is considered as seperate position.

In one-way accounts, we will have only one position that any new trade will increase and any closing trade will decrease the position's volume.

In [ ]:
class Position:
        

    def __init__(self):

        self.containingTrades = []
        self.overallVolume    = 0

        self.ticketPosition   = None

        self.isPositionAlive  = False
        self.isPositionDead   = False

        self.valuePositionPnL = 0
    #



    def GeneratePositionTicket(self):

        return (random.randint(9999, 999999999999))
    #




    def IncreaseTrade(self, newTrade):

        self.containingTrades.append(newTrade)
        self.CanculateOverallVolume(self)

        self.isPositionAlive = True
        self.isPositionDead  = False
    #




    def DecreaseTrade(self, closingTrade):

        self.CalculatePositionPnL()
        self.CanculateOverallVolume()

        self.trades = [trade for trade in self.trades if (trade.ticketTrade != closingTrade.ticketTrade)]

        self.CanculateOverallVolume()


        if (self.overallVolume==0):

            self.KillPosition()
        #
    #




    def KillPosition(self):

        self.isPositionAlive = False
        self.isPositionDead  = True
    #




    def CalculatePositionPnL(self):

        for trade in self.containingTrades:

            self.valuePositionPnL += trade.CalculatePositionPnL()
        #
    #




    def CanculateOverallVolume(self):

        for trade in self.containingTrades:
            
            self.overallVolume += (trade.enumDirection=='BuyLong') if (trade.valueVolume) else (trade.valueVolume*-1) 
        #
    #




    def RefreshStatus(self, aCandleStick: CandleStick):
        
        for trade in self.containingTrades:

            trade.RefreshStatus(aCandleStick)
        #
    #




    def RefreshStatus(self, aTick: Tick):

        for trade in self.containingTrades:

            trade.RefreshStatus(aTick)
        #
    #




    def RefreshStatus(self, aCandleTick: CandleTick):

        for trade in self.containingTrades:

            trade.RefreshStatus(aCandleTick)
        #
    #
#

### ======================================
<h3 style="color:green">Trade Class</h3>

A Trade is an Opened Order.

In [ ]:
class Trade:

    
    def __init__(self, ticketOrder, indexOpened, valueVolume, priceOpened, priceSL=0, priceTP=0, enumDirection=('BuyLong','SellShort')):

        self.ticketOrder   = ticketOrder

        self.indexOpened   = indexOpened
        self.valueVolume   = valueVolume
        self.priceOpened   = priceOpened
        self.priceSL       = priceSL
        self.priceTP       = priceTP
        self.enumDirection = enumDirection

        self.ticketTrade   = None

        self.isTradeAlive  = False
        self.isTradeDead   = False

        self.ClosedPrice   = None

        self.valuePnL      = 0
    #




    def PrintTraded(self):

        print("Trade Ticket: ", self.ticketTrade, "\t", "PnL= ", self.valuePnL)
    #





    def GenerateTradeTicket(self):

        pass
    #




    def KillTrade(self, killingAtPrice):

        self.isTradeAlive = False
        self.isClosed     = True
        self.ClosedPrice  = killingAtPrice

        self.CalculateTradePnL(self.ClosedPrice)
    #
    

    

    def CalculateTradePnL(self, calculatingAtPrice):

        result = 0.0

        if(self.enumDirection=='BuyLong'):

            openedQuality    = (self.valueVolume*self.priceOpened)
            thisPriceQuality = (self.valueVolume*calculatingAtPrice)

            result = (thisPriceQuality-openedQuality)
        #
        elif (self.enumDirection=='SellShort'):

            openedQuality    = (self.valueVolume*self.priceOpened)
            thisPriceQuality = (self.valueVolume*calculatingAtPrice)

            result = (openedQuality-thisPriceQuality)
        #




        self.valuePnL = result
        return (result)
    #



    
    def RefreshStatus(self, aCandleStick: CandleStick):

        if (self.enumDirection=='BuyLong'):

            if (aCandleStick.low <= self.priceSL and self.priceSL!=0):

                self.KillTrade(self.priceSL)
                return
            #
            elif (aCandleStick.high+aCandleStick.slippage >= self.priceTP and self.priceTP!=0):

                self.KillTrade(self.priceTP)
                return
            #

            self.CalculateTradePnL(aCandleStick.low)
        #
        elif (self.enumDirection=='SellShort'):

            if (aCandleStick.high+aCandleStick.slippage >= self.priceSL and self.priceSL!=0):

                self.KillTrade(self.priceSL)
                return
            #
            elif (aCandleStick.low+aCandleStick.slippage <= self.priceTP and self.priceTP!=0):

                self.KillTrade(self.priceTP)
                return
            #

            self.CalculateTradePnL(aCandleStick.high+aCandleStick.slippage)
        #
    #




    def RefreshStatus(self, aTick: Tick):

        if (self.enumDirection=='BuyLong'):

            if (aTick.bidPrice <= self.priceSL and self.priceSL!=0):

                self.KillTrade(aTick.bidPrice)
                return
            #
            elif (aTick.bidPrice >= self.priceTP and self.priceTP!=0):

                self.KillTrade(aTick.bidPrice)
                return
            #

            self.CalculateTradePnL(aTick.bidPrice)
        #
        elif (self.enumDirection=='SellShort'):

            if (aTick.askPrice >= self.priceSL and self.priceSL!=0):

                self.KillTrade(aTick.askPrice)
                return
            #
            elif (aTick.askPrice <= self.priceTP and self.priceTP!=0):

                self.KillTrade(aTick.askPrice)
                return
            #

            self.CalculateTradePnL(aTick.askPrice)
        # 
    #




    def RefreshStatus(self, aCandleTick: CandleTick):

        for aTick in aCandleTick.ticks:

            if (self.enumDirection=='BuyLong'):

                if (aTick.bidPrice <= self.priceSL and self.priceSL!=0):

                    self.KillTrade(aTick.bidPrice)
                    return
                #
                elif (aTick.bidPrice >= self.priceTP and self.priceTP!=0):

                    self.KillTrade(aTick.bidPrice)
                    return
                #

                self.CalculateTradePnL(aTick.bidPrice)
            #
            elif (self.enumDirection=='SellShort'):

                if (aTick.askPrice >= self.priceSL and self.priceSL!=0):

                    self.KillTrade(aTick.askPrice)
                    return
                #
                elif (aTick.askPrice <= self.priceTP and self.priceTP!=0):

                    self.KillTrade(aTick.askPrice)
                    return
                #

                self.CalculateTradePnL(aTick.askPrice)
            # 
        #
    #
#

### ======================================
<h3 style="color:green">Order Class</h3>

An Order is a command that TheSignalerModule sends to the platform.

In [ ]:
class Order:


    def __init__(self, indexSignaled, valueVolume, priceOpen, priceSL=0, priceTP=0, priceTrigger=0, enumDirection=('BuyLong','SellShort'), enumType=('Limit','TriggerLimit','Market','TriggerMarket')):

        self.indexSignaled = indexSignaled


        self.valueVolume      = valueVolume
        self.priceOpen        = priceOpen
        self.priceSL          = priceSL
        self.priceTP          = priceTP
        self.priceTrigger     = priceTrigger
        self.istrigered       = (True) if (enumType=='Limit' or enumType=='Market') else (False)

        self.ticketOrder      = None

        self.enumDirection    = enumDirection
        self.enumType         = enumType

        self.isOrderAlive     = False
        self.isOrderDead      = False
        self.isOrderCancelled = False

        self.isOrderOpened    = False
        self.indexOpened      = None
        self.timestampOpened  = None
        self.priceOpened      = None
    #
    



    def NowOpen(self, indexOpening, timestampOpening, priceOpening):

        self.indexOpened     = indexOpening
        self.timestampOpened = timestampOpening
        self.isOpened        = True
        self.isOrderAlive    = False
        self.isOrderDead     = True
        self.openedPrice     = priceOpening



        newTrade = Trade(self.ticketOrder, self.indexOpened, self.timestampOpened, self.valueVolume, self.openedPrice, self.priceSL, self.priceTP, self.enumDirection)
        
        if (backtesterType=='TwoWay'):
            
            newPosition = Position()
            newPosition.IncreaseTrade(newTrade)
            backtesterPositions.append(newPosition)
        #
        elif (backtesterType=='OneWay'):
            pass
        #
    #
    
    


    def TryOpen(self, aCandlestick: CandleStick):

        if (self.enumDirection=='BuyLong'):


            if (self.enumType=='TrigerLimit'):

                if (self.trigered==True):

                    if (aCandlestick.high + aCandlestick.slippage >= self.priceOpen):

                        self.NowOpen(aCandlestick.index, aCandlestick.timestampOpened, self.priceOpen)
                        return
                    #
                #
                elif (self.trigered==False):

                    if (aCandlestick.high >= self.priceTrigger):

                        self.trigered = True
                        self.TryOpen(aCandlestick)
                        return
                    #
                #
            #
            elif (self.enumType=='TrigerMarket'):

                if (self.trigered==True):

                    self.NowOpen(aCandlestick.index, aCandlestick.timestampOpened, (aCandlestick.low + aCandlestick.slippage))
                    return
                #
                elif (self.trigered==False):

                    if (aCandlestick.high >= self.priceTrigger):

                        self.trigered = True
                        self.TryOpen(aCandlestick)
                        return
                    #
                #
            #
            elif (self.enumType=='Market'):

                self.NowOpen(aCandlestick.index, aCandlestick.timestampOpened, (aCandlestick.low + aCandlestick.slippage))
                return
            #
            elif (self.enumType=='Limit'):
                    
                if (aCandlestick.high + aCandlestick.slippage >= self.priceOpen):

                    self.NowOpen(aCandlestick.index, aCandlestick.timestampOpened, self.priceOpen)
                    return
                #
            #
        #
        elif (self.enumDirection=='SellShort'):


            if (self.enumType=='TrigerLimit'):

                if (self.trigered==True):

                    if (aCandlestick.low <= self.priceOpen):

                        self.NowOpen(aCandlestick.index, aCandlestick.timestampOpened, self.priceOpen)
                        return
                    #
                #
                elif (self.trigered==False):

                    if (aCandlestick.low <= self.priceTrigger):

                        self.trigered = True
                        self.TryOpen(aCandlestick)
                        return
                    #
                #
            #
            elif (self.enumType=='TrigerMarket'):

                if (self.trigered==True):

                    self.NowOpen(aCandlestick.index, aCandlestick.timestampOpened, (aCandlestick.low))
                    return
                #
                elif (self.trigered==False):

                    if (aCandlestick.low <= self.priceTrigger):

                        self.trigered = True
                        self.TryOpen(aCandlestick)
                        return
                    #
                #
            #
            elif (self.enumType=='Market'):

                self.NowOpen(aCandlestick.index, aCandlestick.timestampOpened, (aCandlestick.low))
                return
            #
            elif (self.enumType=='Limit'):
                    
                if (aCandlestick.low <= self.priceOpen):

                    self.NowOpen(aCandlestick.index, aCandlestick.timestampOpened, self.priceOpen)
                    return
                #
            #
        #
    #




    def TryOpen(self, aTick: Tick):

        if (self.enumDirection=='BuyLong'):


            if (self.enumType=='TrigerLimit'):

                if (self.trigered==True):

                    if (aTick.askPrice>= self.priceOpen):

                        self.NowOpen(aTick.index, aTick.timestamp, aTick.askPrice)
                        return
                    #
                #
                elif (self.trigered==False):

                    if (aTick.tradedprice >= self.priceTrigger):

                        self.trigered = True
                        self.TryOpen(aTick)
                        return
                    #
                #
            #
            elif (self.enumType=='TrigerMarket'):

                if (self.trigered==True):

                    self.NowOpen(aTick.index, aTick.timestamp, aTick.askPrice)
                    return
                #
                elif (self.trigered==False):

                    if (aTick.tradedprice >= self.priceTrigger):

                        self.trigered = True
                        self.TryOpen(aTick)
                        return
                    #
                #
            #
            elif (self.enumType=='Market'):

                self.NowOpen(aTick.index, aTick.timestamp, aTick.askPrice)
                return
            #
            elif (self.enumType=='Limit'):

                if (aTick.askPrice >= self.priceOpen):

                    self.NowOpen(aTick.index, aTick.timestamp, aTick.askPrice)
                    return
                #
            #
        #
        elif (self.enumDirection=='SellShort'):


            if (self.enumType=='TrigerLimit'):

                if (self.trigered==True):

                    if (aTick.bidPrice <= self.priceOpen):

                        self.NowOpen(aTick.index, aTick.timestamp, aTick.bidPrice)
                        return
                    #
                #
                elif (self.trigered==False):

                    if (aTick.tradedprice <= self.priceTrigger):

                        self.trigered = True
                        self.TryOpen(aTick)
                        return
                    #
                #
            #
            elif (self.enumType=='TrigerMarket'):

                if (self.trigered==True):

                    self.NowOpen(aTick.index, aTick.timestamp, aTick.bidPrice)
                    return
                #
                elif (self.trigered==False):

                    if (aTick.tradedprice <= self.priceTrigger):

                        self.trigered = True
                        self.TryOpen(aTick)
                        return
                    #
                #
            #
            elif (self.enumType=='Market'):

                self.NowOpen(aTick.index, aTick.timestamp, aTick.bidPrice)
                return
            #
            elif (self.enumType=='Limit'):

                if (aTick.bidPrice <= self.priceOpen):

                    self.NowOpen(aTick.index, aTick.timestamp, aTick.bidPrice)
                    return
                #
            #
        #
    #




    def TryOpen(self, aCandleTick: CandleTick):

        for aTick in aCandleTick.ticks:

            if (self.enumDirection=='BuyLong'):


                if (self.enumType=='TrigerLimit'):

                    if (self.trigered==True):

                        if (aTick.askPrice>= self.priceOpen):

                            self.NowOpen(aTick.index, aTick.timestamp, aTick.askPrice)
                            return
                        #
                    #
                    elif (self.trigered==False):

                        if (aTick.tradedprice >= self.priceTrigger):

                            self.trigered = True
                            self.TryOpen(aTick)
                            return
                        #
                    #
                #
                elif (self.enumType=='TrigerMarket'):

                    if (self.trigered==True):

                        self.NowOpen(aTick.index, aTick.timestamp, aTick.askPrice)
                        return
                    #
                    elif (self.trigered==False):

                        if (aTick.tradedprice >= self.priceTrigger):

                            self.trigered = True
                            self.TryOpen(aTick)
                            return
                        #
                    #
                #
                elif (self.enumType=='Market'):

                    self.NowOpen(aTick.index, aTick.timestamp, aTick.askPrice)
                    return
                #
                elif (self.enumType=='Limit'):

                    if (aTick.askPrice >= self.priceOpen):

                        self.NowOpen(aTick.index, aTick.timestamp, aTick.askPrice)
                        return
                    #
                #
            #
            elif (self.enumDirection=='SellShort'):


                if (self.enumType=='TrigerLimit'):

                    if (self.trigered==True):

                        if (aTick.bidPrice <= self.priceOpen):

                            self.NowOpen(aTick.index, aTick.timestamp, aTick.bidPrice)
                            return
                        #
                    #
                    elif (self.trigered==False):

                        if (aTick.tradedprice <= self.priceTrigger):

                            self.trigered = True
                            self.TryOpen(aTick)
                            return
                        #
                    #
                #
                elif (self.enumType=='TrigerMarket'):

                    if (self.trigered==True):

                        self.NowOpen(aTick.index, aTick.timestamp, aTick.bidPrice)
                        return
                    #
                    elif (self.trigered==False):

                        if (aTick.tradedprice <= self.priceTrigger):

                            self.trigered = True
                            self.TryOpen(aTick)
                            return
                        #
                    #
                #
                elif (self.enumType=='Market'):

                    self.NowOpen(aTick.index, aTick.timestamp, aTick.bidPrice)
                    return
                #
                elif (self.enumType=='Limit'):

                    if (aTick.bidPrice <= self.priceOpen):

                        self.NowOpen(aTick.index, aTick.timestamp, aTick.bidPrice)
                        return
                    #
                #
            #
        #
    #




    def CancelOrder(self):

        self.indexOpened      = 0
        self.timestampOpened  = 0
        self.isOrderAlive     = False
        self.isOrderDead      = True
        self.isOrderCancelled = True
        self.openedPrice      = 0
    #
#

### ======================================
<h3 style="color:green">Indicator Class</h3>

An Indicator is a data processor that indicates and shows the core logic of market conditions.

It can be applies to two datas:    1) Market data    2) Another indicatored data.


Parameters:

<li><b>dataMarket</b>: Raw Data to be processed on.</li>

<li><b>dataIndicator</b>: Indicatored Data to be processed on.</li>

<li><b>countBuffers</b>: Data containers for the drawing datas.</li>

<li><b>typeDraw</b>: Line, Arrow, to be developed more types...</li>

<li><b>typeWindow</b>: The indicator chart to be int seperate window or in the market chart.</li>

In [ ]:
class Indicator:


    def __init__(self, applyingData:pd.DataFrame, countBuffers:int, typeWindow=('SamePanel','SeperatePanel','SeperateChart')):

        self.applyingData = applyingData
        self.countBuffers = countBuffers
        self.typeWindow   = typeWindow

        self.times   = self.applyingData['time'].to_numpy()
        self.opens   = self.applyingData['open'].to_numpy()
        self.highs   = self.applyingData['high'].to_numpy()
        self.lows    = self.applyingData['low'].to_numpy()
        self.closes  = self.applyingData['close'].to_numpy()
        self.spreads = self.applyingData['spread'].to_numpy()
        self.volumes = self.applyingData['volume'].to_numpy()
        self.times.flags.writeable   = False
        self.opens.flags.writeable   = False
        self.highs.flags.writeable   = False
        self.lows.flags.writeable    = False
        self.closes.flags.writeable  = False
        self.spreads.flags.writeable = False
        self.volumes.flags.writeable = False


        self.buffersList = []
        for n in range(countBuffers):

            self.buffersList.append(np.full(len(applyingData), np.nan))
        #
    #




    def SetBufferType(self, numberBuffer, nameBuffer, digitsBuffer, typeDrawBuffer=('MiddleCalculations','None','Line','Arrow','Histogram1','Histogram2','Section','ZigZag','Filling','Bars','Candles')):
                  
        pass
    #




    def SetBufferMeanlessValue(self, whichBuffer:int, newName:str):

        self.data.rename(columns={'Buffer_'+str(whichBuffer) : newName}, inplace=True)
    #




    def MainIterator(self):

        for bufferNum in range(len(self.buffersList)):

            self.buffersList[bufferNum] = talib.SMA(self.closes, timeperiod=10*(bufferNum+1))
        #
    #
   
   


    def Indicate(self):

        self.MainIterator()
    #
   
   
   
       
    def Indicated(self):

        theDF = pd.DataFrame()

        theDF['time']   = self.times
        theDF['open']   = self.opens
        theDF['high']   = self.highs
        theDF['low']    = self.lows
        theDF['close']  = self.closes
        theDF['spread'] = self.spreads
        theDF['volume'] = self.volumes

        for bufferNum in range(len(self.buffersList)):

            theDF['Buffer_'+str(bufferNum)] = pd.Series(self.buffersList[bufferNum])
        #


        return (theDF)
    # 




    def PrintData(self):

        print(self.data)
    #
#

### ======================================
<h3 style="color:blue">Apply</h3>

An Order is a command that TheSignalerModule sends to the platform.

In [ ]:
df = pd.read_csv('./0_RawChartData/Crypto_Binance_BTCUSDT.P_5min.csv')


df['time'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'])
cols = ['time'] + [col for col in df.columns if col != 'time']
df = df[cols]

df.drop(columns=['DATE', 'TIME', 'TICKVOL'], inplace=True)
df.rename(columns={'time':'time', 'OPEN':'open', 'HIGH':'high', 'LOW':'low', 'CLOSE':'close', 'VOL':'volume', 'SPREAD':'spread'}, inplace=True)


# df = df.head(1000)
df

In [ ]:
indicatorSMA = Indicator(df, 2, 'SamePanel')
indicatorSMA.Indicate()
processed = indicatorSMA.Indicated()

In [ ]:
chart = JupyterChart(inner_width=0.5, inner_height=0.5, width=1300, height=500, toolbox=True)
chart.set(processed)

In [ ]:
for buffersCount in range(indicatorSMA.countBuffers):

    buffer  = chart.create_line(name='Buffer_'+str(buffersCount), color='rgba(255, 0, 0, 0.6)', price_label=False, price_line=False)
    buffer.set(processed)
#

In [ ]:
chart.load()